## Case 2

In [74]:
import sys
sys.path.append('C:\\Users\\INDIA\\Rahul_work\\functions')
import pandas as pd
from datetime import datetime
from stock_data import fetch_stock_data
from determine_entry_point import process_stock_data
from tvDatafeed import TvDatafeed,Interval
import time
import math


In [78]:
def fetch_case2_satisfied_symbols(symbols_path, retry_delay=1):
    symbols = pd.read_csv(symbols_path)

    satisfied_case2_symbols = pd.DataFrame(columns=['symbol', 'price_at_scan', 'high', 'low', 'volume_at_scan', 'avg_31_day_vol'])
    
    for symbol in symbols.symbol:
        fifteen_min_data = None
        today_data = None

        while fifteen_min_data is None or fifteen_min_data.empty:
            try:
                fifteen_min_data = fetch_stock_data(symbol, interval=Interval.in_15_minute, n_bars=775)
                if fifteen_min_data is None:
                    raise ValueError("Fetched data is None")
            except Exception as e:
                print(f"Error fetching data for {symbol}: {e}")
                time.sleep(retry_delay)
                continue

        today_date = str(datetime.today().date())
        filtered_df = fifteen_min_data[~(fifteen_min_data.index.date == pd.to_datetime(today_date).date())] # remove today's 15min data 
        selected_rows = filtered_df.between_time('09:15:00', '10:30:00') 
        daily_volumes = selected_rows.groupby(selected_rows.index.date)['volume'].sum()
        avg_31 = daily_volumes.mean()
        more_than_30_percent = avg_31 + (0.3 * avg_31)

        while today_data is None:
            try:
                today_data = fetch_stock_data(symbol, interval=Interval.in_daily, n_bars=1)
                if today_data is None:
                    raise ValueError("Fetched data is None")
            except Exception as e:
                print(f"Error fetching daily data for {symbol}: {e}")
                time.sleep(retry_delay)
                continue
        
        today_close = today_data['close'][0]
        today_high = today_data['high'][0]
        today_low = today_data['low'][0]
        today_vol = today_data['volume'][0]
        
        if today_close > 5000:
            continue
            
        today_perc_change = abs((today_data['close'][0] - today_data['open'][0]) / today_data['open'][0] * 100)
        
        if more_than_30_percent < today_vol and today_perc_change < 2:
            transaction_type = ''
            if today_data['open'][0] < today_close:
                transaction_type = 'BUY'
            else:
                transaction_type = 'SELL'
                
            new_row = pd.DataFrame({
                'symbol': [symbol], 
                'price_at_scan': [today_close],
                'high': [today_high],
                'low': [today_low],
                'volume_at_scan': [today_vol],
                'avg_31_day_vol': [round(avg_31)],
                'type': [transaction_type]
            })
            satisfied_case2_symbols = pd.concat([satisfied_case2_symbols, new_row], ignore_index=True)
 
    return satisfied_case2_symbols

In [67]:
symbols_path = "Preparing data/symbols.csv"
satisfied_case2_symbols = fetch_case2_satisfied_symbols(symbols_path)

ERROR:tvDatafeed.main:Connection is already closed.
ERROR:tvDatafeed.main:no data, please check the exchange and symbol


Error fetching daily data for ADANIENT: Fetched data is None


ERROR:tvDatafeed.main:Connection is already closed.
ERROR:tvDatafeed.main:no data, please check the exchange and symbol


Error fetching daily data for BAJAJ_AUTO: Fetched data is None


ERROR:tvDatafeed.main:Connection is already closed.
ERROR:tvDatafeed.main:no data, please check the exchange and symbol


Error fetching data for BPCL: Fetched data is None


In [68]:
satisfied_case2_symbols

,symbol,price_at_scan,high,low,volume_at_scan,avg_31_day_vol,type
0,ASIANPAINT,2910.50,2927.00,2886.20,1145447.0,459416,BUY
1,BHARTIARTL,1440.65,1475.60,1438.50,2885552.0,2036133,SELL
2,CIPLA,1504.65,1511.30,1485.00,1961457.0,491690,BUY
3,COALINDIA,495.30,499.40,485.10,5255085.0,3075403,BUY
4,DIVISLAB,4520.50,4579.30,4501.45,293596.0,145002,BUY
5,EICHERMOT,4904.90,4942.30,4879.00,271683.0,170802,SELL
6,HCLTECH,1581.15,1598.00,1576.85,1724820.0,1268136,SELL
7,HDFCBANK,1597.55,1621.00,1588.05,20883998.0,6915846,SELL
8,HINDUNILVR,2711.55,2757.45,2675.10,3018757.0,549761,SELL
9,ICICIBANK,1212.25,1233.65,1202.05,12663010.0,3935464,SELL


In [75]:
process_stock_data(satisfied_case2_symbols)  

,symbol,price_at_scan,high,low,volume_at_scan,avg_31_day_vol,type,range
0,ASIANPAINT,2910.50,2927.00,2886.20,1145447.0,459416,BUY,2927.00
1,BHARTIARTL,1440.65,1475.60,1438.50,2885552.0,2036133,SELL,1438.50
2,CIPLA,1504.65,1511.30,1485.00,1961457.0,491690,BUY,1511.30
3,COALINDIA,495.30,499.40,485.10,5255085.0,3075403,BUY,500.00
4,DIVISLAB,4520.50,4579.30,4501.45,293596.0,145002,BUY,4600.00
5,EICHERMOT,4904.90,4942.30,4879.00,271683.0,170802,SELL,4879.00
6,HCLTECH,1581.15,1598.00,1576.85,1724820.0,1268136,SELL,1576.85
7,HDFCBANK,1597.55,1621.00,1588.05,20883998.0,6915846,SELL,1588.05
8,HINDUNILVR,2711.55,2757.45,2675.10,3018757.0,549761,SELL,2675.10
9,ICICIBANK,1212.25,1233.65,1202.05,12663010.0,3935464,SELL,1200.00


In [76]:
satisfied_case2_symbols.to_csv('case2.csv', index=False)